In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from helper import get_all_csv_files_path, calculate_statistics, calculate_all_feature_product
from generate_adaptive_boundaries_optimization import run_ga, calculate_features_for_boundaries
from find_correct_clusters_and_do_ks_test import elbow_method, generate_train_test_indices, plot_kde_train_test, perform_ks_test
import shap



In [2]:
data_folder = '../Data/'
filename = data_folder + 'spring24_solvtemp_jpm.csv'
df = pd.read_csv(filename)


Data Splitting

In [ ]:
normalized_df = elbow_method(df)
num_clusters = 5 # Set this based on elbow method results
train_indices, val_indices,  test_indices = generate_train_test_indices(num_clusters, normalized_df)
print(train_indices)
print(val_indices)
print(test_indices)
plot_kde_train_test(train_indices, val_indices, test_indices, df)
ks_results = perform_ks_test(train_indices, val_indices, test_indices, df)
print(ks_results)

**Expert Model**

Model 4 - E-QSPR 

Input - Expert Defined Features

Output - Conductivity

In [ ]:
input_columns = ['E 00',	'E 01',	'E 02',	'00/01',	'Relative Anion Signal',	'Bleaching %',	'polaron ratio']
input_data = df[input_columns].values
output_data = df[['conductivity']].values
lower_limit = np.array([0, 0, 0, 30])
upper_limit = np.array([100, 100, 100, 270])
X_train = input_data[train_indices]
y_train = output_data[train_indices]
X_val = input_data[val_indices]
y_val = output_data[val_indices]
X_test = input_data[test_indices]
y_test = output_data[test_indices]


model_4 = GradientBoostingRegressor(loss ='squared_error', learning_rate=0.1, n_estimators=100, min_samples_leaf=1, random_state=42)
 
y_pred_train, y_pred_val, y_pred_test,_,_,_,_,_,_,_,_,_,_,_,_,_,_,_ = calculate_statistics(model_4, X_train, y_train, X_val, y_val, X_test, y_test)

plt.figure(figsize=(6, 6))
plt.scatter(y_train, y_pred_train, color='black', alpha=0.7, label='Train')
plt.scatter(y_val, y_pred_val, color='blue', alpha=0.7, label='Val')
plt.scatter(y_test, y_pred_test, color='red', alpha=0.7, label='Test')
plt.plot([min(np.concatenate((y_train, y_test))), max(np.concatenate((y_train, y_test)))],
    [min(np.concatenate((y_train, y_test))), max(np.concatenate((y_train, y_test)))],
    color='green', linestyle='--')
plt.xlabel('True Conductivity (S/cm)', fontsize=24)
plt.ylabel('Predicted Conductivity(S/cm)', fontsize=24)
plt.legend(fontsize=24)
plt.tight_layout()
plt.xticks(fontsize=24)
plt.yticks(fontsize=24)
plt.show()


**Data-Driven Models**

In [6]:
n_preanneal_boundaries=5
preanneal_boundaries_limit = [300, 900]
n_postanneal_boundaries=5
postanneal_boundaries_limit = [300, 900]
n_postdope_boundaries=5
postdope_boundaries_limit = [300, 2450]

In [7]:
measurement_type = 'Postanneal'
mode = "train"
spectra_axis = 'energy'  # 'energy' or 'wavelength'
on_second_order = True
if measurement_type == 'Preanneal':
    n_boundaries = n_preanneal_boundaries
    boundaries_limit = preanneal_boundaries_limit
elif measurement_type == 'Postanneal':
    n_boundaries = n_postanneal_boundaries
    boundaries_limit = postanneal_boundaries_limit
elif measurement_type == 'Postdope':
    n_boundaries = n_postdope_boundaries
    boundaries_limit = postdope_boundaries_limit

if spectra_axis == 'energy':
    boundaries_limit = [1239.8 / boundary for boundary in boundaries_limit]
    boundaries_limit = boundaries_limit[::-1] 

measurement_csv_files_train, measurement_csv_files_val, measurement_csv_files_test = get_all_csv_files_path(data_folder, measurement_type, train_indices, val_indices, test_indices)

if mode == "train":
    measurement_files = measurement_csv_files_train
else:
    measurement_files = measurement_csv_files_test

Genetic Algorithm

In [ ]:
loss_progress = []
order = 2
optimal_boundaries, result, loss_progress = run_ga(measurement_files, df, boundaries_limit, 
                                                   n_boundaries, spectra_axis,  order, 
                                                   n = 100,  ngen=100)

In [8]:

optimal_boundaries= [1.3776, 1.8284, 1.9825, 2.0952, 2.7003]  # Identified using GA on Postanneal Spectra Data


Model 1 - I-QSPR 1

Input - Spectra Features + Processing Conditions

Output -  Conductivity

In [ ]:
measurement_csv_files_train, measurement_csv_files_val, measurement_csv_files_test = get_all_csv_files_path(data_folder, measurement_type, train_indices, val_indices, test_indices)


# train
df_measurement_features_train_orignal = calculate_features_for_boundaries(optimal_boundaries, measurement_csv_files_train, spectra_axis='energy',  order=0)
feature_names = []
for i in range(len(optimal_boundaries) - 1):
    feature_name = f"AUC_{i+1}"
    feature_names.append(feature_name)


df_measurement_features_train_orignal.columns = ['exp_id'] + feature_names

df_measurement_features_train_derivative = calculate_features_for_boundaries(optimal_boundaries, measurement_csv_files_train, spectra_axis='energy',  order=2)
feature_names_derivative = []
for i in range(len(optimal_boundaries) - 1):
    feature_name = f"d\u00B2AUC_{i+1}"
    feature_names_derivative.append(feature_name)
df_measurement_features_train_derivative.columns = ['exp_id'] + feature_names_derivative

df_measurement_features_train = pd.merge(df_measurement_features_train_orignal, df_measurement_features_train_derivative, on='exp_id', how='inner')
df_measurement_features_train = pd.merge(df_measurement_features_train, df[['exp_id', 'CB', 'DCB', 'TOL', 'annealing_temperature', 'conductivity']], on='exp_id', how='inner')

#val
df_measurement_features_val_orignal = calculate_features_for_boundaries( optimal_boundaries, measurement_csv_files_val, spectra_axis='energy',  order=0)
df_measurement_features_val_orignal.columns = ['exp_id'] + feature_names

df_measurement_features_val_derivative = calculate_features_for_boundaries( optimal_boundaries, measurement_csv_files_val, spectra_axis='energy',  order=2)
df_measurement_features_val_derivative.columns = ['exp_id'] + feature_names_derivative

df_measurement_features_val = pd.merge(df_measurement_features_val_orignal, df_measurement_features_val_derivative, on='exp_id', how='inner')
df_measurement_features_val = pd.merge(df_measurement_features_val, df[['exp_id', 'CB', 'DCB', 'TOL', 'annealing_temperature', 'conductivity']], on='exp_id', how='inner')

#test
df_measurement_features_test_orignal = calculate_features_for_boundaries( optimal_boundaries, measurement_csv_files_test, spectra_axis='energy',  order=0)
df_measurement_features_test_orignal.columns = ['exp_id'] + feature_names

df_measurement_features_test_derivative = calculate_features_for_boundaries( optimal_boundaries, measurement_csv_files_test, spectra_axis='energy',  order=2)
df_measurement_features_test_derivative.columns = ['exp_id'] + feature_names_derivative

df_measurement_features_test = pd.merge(df_measurement_features_test_orignal, df_measurement_features_test_derivative, on='exp_id', how='inner')
df_measurement_features_test = pd.merge(df_measurement_features_test, df[['exp_id', 'CB', 'DCB', 'TOL', 'annealing_temperature', 'conductivity']], on='exp_id', how='inner')



X_train = df_measurement_features_train.drop(columns=['exp_id', 'conductivity'])  # All columns except exp_id and conductivity
y_train = df_measurement_features_train['conductivity'].values

X_val = df_measurement_features_val.drop(columns=['exp_id', 'conductivity'])
y_val = df_measurement_features_val['conductivity'].values

X_test = df_measurement_features_test.drop(columns=['exp_id', 'conductivity'])
y_test = df_measurement_features_test['conductivity'].values

model_1 = RandomForestRegressor(n_estimators=70, criterion = "squared_error",  min_samples_split=5, random_state=42)

y_pred_train, y_pred_val, y_pred_test, _, _, _, _, _, _,_, _,_,_, _, _ ,_, _, _ = calculate_statistics(model_1, X_train, y_train, X_val, y_val, X_test, y_test)


plt.figure(figsize=(6.2, 6.2))
plt.scatter(y_train, y_pred_train, color='black', alpha=0.7, label='Train')
plt.scatter(y_val, y_pred_val, color='blue', alpha=0.7, label='Val')
plt.scatter(y_test, y_pred_test, color='red', alpha=0.7, label='Test')
plt.plot([min(np.concatenate((y_train, y_test))), max(np.concatenate((y_train, y_test)))],
    [min(np.concatenate((y_train, y_test))), max(np.concatenate((y_train, y_test)))],
    color='green', linestyle='--')
plt.xlabel('True Conductivity (S/cm)', fontsize=24)
plt.ylabel('Predicted Conductivity(S/cm)', fontsize=24)
plt.legend(fontsize=24)
plt.tight_layout()
plt.xticks(fontsize=24)
plt.yticks(fontsize=24)
plt.show()

Model 2 - I-QSPR 2

Input - Spectra Features + Processing Conditions + Product of Spectra Features

Output -  Conductivity

In [ ]:
measurement_csv_files_train, measurement_csv_files_val, measurement_csv_files_test = get_all_csv_files_path(data_folder, measurement_type, train_indices, val_indices, test_indices)

# Train Data
df_measurement_features_train_orignal = calculate_features_for_boundaries(optimal_boundaries, measurement_csv_files_train, spectra_axis='energy',  order=0)
feature_names = []
for i in range(len(optimal_boundaries) - 1):
    feature_name = f"AUC_{i+1}"
    feature_names.append(feature_name)
df_measurement_features_train_orignal.columns = ['exp_id'] + feature_names
df_measurement_features_train_derivative = calculate_features_for_boundaries(optimal_boundaries, measurement_csv_files_train, spectra_axis='energy',  order=2)
feature_names_derivative = []
for i in range(len(optimal_boundaries) - 1):
    feature_name = f"d\u00B2AUC_{i+1}"
    feature_names_derivative.append(feature_name)
df_measurement_features_train_derivative.columns = ['exp_id'] + feature_names_derivative
df_measurement_features_train = pd.merge(df_measurement_features_train_orignal, df_measurement_features_train_derivative, on='exp_id', how='inner')
df_measurement_features_train_ratio = calculate_all_feature_product(df_measurement_features_train)
df_measurement_features_train = pd.merge(df_measurement_features_train, df_measurement_features_train_ratio, on='exp_id', how='inner')
df_measurement_features_train = pd.merge(df_measurement_features_train, df[['exp_id', 'CB', 'DCB', 'TOL', 'annealing_temperature', 'conductivity']], on='exp_id', how='inner')

# Validation Data
df_measurement_features_val_orignal = calculate_features_for_boundaries( optimal_boundaries, measurement_csv_files_val, spectra_axis='energy',  order=0)
df_measurement_features_val_orignal.columns = ['exp_id'] + feature_names
df_measurement_features_val_derivative = calculate_features_for_boundaries( optimal_boundaries, measurement_csv_files_val, spectra_axis='energy',  order=2)
df_measurement_features_val_derivative.columns = ['exp_id'] + feature_names_derivative
df_measurement_features_val = pd.merge(df_measurement_features_val_orignal, df_measurement_features_val_derivative, on='exp_id', how='inner')
df_measurement_features_val_ratio = calculate_all_feature_product(df_measurement_features_val)
df_measurement_features_val = pd.merge(df_measurement_features_val, df_measurement_features_val_ratio, on='exp_id', how='inner')
df_measurement_features_val = pd.merge(df_measurement_features_val, df[['exp_id', 'CB', 'DCB', 'TOL', 'annealing_temperature',  'conductivity']], on='exp_id', how='inner')

# Test Data
df_measurement_features_test_orignal = calculate_features_for_boundaries( optimal_boundaries, measurement_csv_files_test, spectra_axis='energy',  order=0)
df_measurement_features_test_orignal.columns = ['exp_id'] + feature_names
df_measurement_features_test_derivative = calculate_features_for_boundaries( optimal_boundaries, measurement_csv_files_test, spectra_axis='energy',  order=2)
df_measurement_features_test_derivative.columns = ['exp_id'] + feature_names_derivative
df_measurement_features_test = pd.merge(df_measurement_features_test_orignal, df_measurement_features_test_derivative, on='exp_id', how='inner')
df_measurement_features_test_ratio = calculate_all_feature_product(df_measurement_features_test)
df_measurement_features_test = pd.merge(df_measurement_features_test, df_measurement_features_test_ratio, on='exp_id', how='inner')
df_measurement_features_test = pd.merge(df_measurement_features_test, df[['exp_id', 'CB', 'DCB', 'TOL', 'annealing_temperature',  'conductivity']], on='exp_id', how='inner')

X_train = df_measurement_features_train.drop(columns=['exp_id', 'conductivity'])  
y_train = df_measurement_features_train['conductivity'].values

X_val = df_measurement_features_val.drop(columns=['exp_id', 'conductivity'])
y_val = df_measurement_features_val['conductivity'].values

X_test = df_measurement_features_test.drop(columns=['exp_id', 'conductivity'])
y_test = df_measurement_features_test['conductivity'].values

model_2 = RandomForestRegressor(n_estimators=50, criterion = "squared_error",  min_samples_split=2,  random_state=42)
y_pred_train, y_pred_val, y_pred_test, _, _, _, _, _, _,_, _,_,_, _, _, _, _, _ = calculate_statistics(model_2, X_train, y_train, X_val, y_val, X_test, y_test)


plt.figure(figsize=(6.2, 6.2))
plt.scatter(y_train, y_pred_train, color='black', alpha=0.7, label='Train')
plt.scatter(y_val, y_pred_val, color='blue', alpha=0.7, label='Val')
plt.scatter(y_test, y_pred_test, color='red', alpha=0.7, label='Test')
plt.plot([min(np.concatenate((y_train, y_test))), max(np.concatenate((y_train, y_test)))],
    [min(np.concatenate((y_train, y_test))), max(np.concatenate((y_train, y_test)))],
    color='green', linestyle='--')
plt.xlabel('True Conductivity (S/cm)', fontsize=24)
plt.ylabel('Predicted Conductivity(S/cm)', fontsize=24)
plt.legend(fontsize=24)
plt.tight_layout()
plt.xticks(fontsize=24)
plt.yticks(fontsize=24)
plt.show()






Using SHAPLEY for Feature Ranking and Forward Selection

Model 3 - I-QSPR 3

Input - Best Features From Spectra Features + Processing Conditions + Product of Subset Features

Output -  Conductivity

In [ ]:
explainer = shap.Explainer(model_2, X_train)
shap_values = explainer(X_train)
# Summary plot
plt.rcParams.update({
    'font.size': 24,
    'axes.labelsize': 24,
    'xtick.labelsize': 24,
    'ytick.labelsize': 24,
    'legend.fontsize': 24,
    'figure.titlesize': 24
})

shap.summary_plot(shap_values, X_train, plot_type="bar", max_display=13, color='k', plot_size=(9,6), show=False)
plt.show()
plt.close()


shap.plots.beeswarm(shap_values, max_display=14, show = False, plot_size=(8,12))
plt.tight_layout()
plt.show()
plt.close()

mean_abs_shap = np.abs(shap_values.values).mean(axis=0)
shap_feature_importance = list(zip(X_test.columns, mean_abs_shap))
shap_feature_importance = sorted(shap_feature_importance, key=lambda x: x[1], reverse=True)
shap_ranked_features = [f[0] for f in shap_feature_importance]

# Forward Selection Method
r2_scores_shap = []
num_features_shap = []
mae_scores_shap = []

for i in range(1, len(shap_ranked_features) + 1):
    selected_features = shap_ranked_features[:i]
    print(f"Using top {i} SHAP features: {selected_features}")
    X_train_subset = X_train[selected_features]
    X_val_subset = X_val[selected_features]
    X_test_subset = X_test[selected_features]
    y_pred_train, y_pred_val,y_pred_test,  _, r2_val, _, _, rmse_val, _, _, mae_val, _, _, _, _, _, _, _ = calculate_statistics(
        model_2, X_train_subset, y_train, X_val_subset, y_val, X_test_subset, y_test
    )
    r2_scores_shap.append(r2_val)
    num_features_shap.append(i)
    mae_scores_shap.append(mae_val)
    if i == 13:  # Max R2 at 13 features
        plt.figure(figsize=(6.5, 6.5))
        plt.scatter(y_train, y_pred_train, color='black', alpha=0.7, label='Train')
        plt.scatter(y_val, y_pred_val, color='blue', alpha=0.7, label='Val')
        plt.scatter(y_test, y_pred_test, color='red', alpha=0.7, label='Test')
        plt.plot([min(np.concatenate((y_train, y_test))), max(np.concatenate((y_train, y_test)))],
            [min(np.concatenate((y_train, y_test))), max(np.concatenate((y_train, y_test)))],
            color='green', linestyle='--')
        plt.xlabel('True Conductivity (S/cm)', fontsize=22)
        plt.ylabel('Predicted Conductivity(S/cm)', fontsize=24)
        plt.legend(fontsize=24)
        plt.tight_layout()
        plt.xticks(fontsize=24)
        plt.yticks(fontsize=24)
        plt.show()

    print("-------------------------------------------------------------")

# Plot MAE vs. number of SHAP-ranked features
plt.figure(figsize=(6, 6))
plt.plot(num_features_shap, mae_scores_shap,  marker='o', color='k')
plt.ylabel('MAE (Val)', fontsize=24)
plt.xlabel('Number of Features', fontsize=24)
plt.xticks([0, 10, 20,  30,  40], fontsize=24)

plt.tight_layout()
plt.grid(True, linestyle='--', alpha=0.5)
plt.show()



**Combined Model**

Using SHAPLEY for Feature Ranking and Forward Selection

Use Best Data Driven Features and Expert Features

Model 5 - QSPR 

Input - Best Features From Spectra Features + Processing Conditions + Product of Subset Features

Output -  Conductivity

In [18]:
data_driven_features = ['d²AUC_2', 'AUC_4*d²AUC_4', 'AUC_4', 'd²AUC_1', 'd²AUC_3', 'AUC_3', 'DCB', 'AUC_4*d²AUC_3', 'd²AUC_4', 'annealing_temperature', 'CB', 'AUC_4*d²AUC_2', 'AUC_2*AUC_4']  # Top 13 features from SHAP analysis
expert_features = ['E 00',	'E 01',	'E 02',	'00/01',	'Relative Anion Signal',	'Bleaching %',	'polaron ratio'] # 7 expert features


Combine model with both features and do shap based feature selection

In [ ]:
measurement_csv_files_train, measurement_csv_files_val, measurement_csv_files_test = get_all_csv_files_path(data_folder, measurement_type, train_indices, val_indices, test_indices)


# Train Data
df_measurement_features_train_orignal = calculate_features_for_boundaries(optimal_boundaries, measurement_csv_files_train, spectra_axis='energy',  order=0)
feature_names = []
for i in range(len(optimal_boundaries) - 1):
    feature_name = f"AUC_{i+1}"
    feature_names.append(feature_name)
df_measurement_features_train_orignal.columns = ['exp_id'] + feature_names
df_measurement_features_train_derivative = calculate_features_for_boundaries(optimal_boundaries, measurement_csv_files_train, spectra_axis='energy',  order=2)
feature_names_derivative = []
for i in range(len(optimal_boundaries) - 1):
    feature_name = f"d\u00B2AUC_{i+1}"
    feature_names_derivative.append(feature_name)
df_measurement_features_train_derivative.columns = ['exp_id'] + feature_names_derivative
df_measurement_features_train = pd.merge(df_measurement_features_train_orignal, df_measurement_features_train_derivative, on='exp_id', how='inner')
df_measurement_features_train_ratio = calculate_all_feature_product(df_measurement_features_train)
df_measurement_features_train = pd.merge(df_measurement_features_train, df_measurement_features_train_ratio, on='exp_id', how='inner')
df_measurement_features_train = pd.merge(df_measurement_features_train, df[['exp_id', 'CB', 'DCB', 'TOL', 'annealing_temperature', 'E 00',	'E 01',	'E 02',	'00/01',	'Relative Anion Signal',	'Bleaching %',	'polaron ratio','conductivity']], on='exp_id', how='inner')
df_measurement_features_train = df_measurement_features_train[data_driven_features + expert_features + ['exp_id', 'conductivity']]

# Validation Data
df_measurement_features_val_orignal = calculate_features_for_boundaries( optimal_boundaries, measurement_csv_files_val, spectra_axis='energy',  order=0)
df_measurement_features_val_orignal.columns = ['exp_id'] + feature_names
df_measurement_features_val_derivative = calculate_features_for_boundaries( optimal_boundaries, measurement_csv_files_val, spectra_axis='energy',  order=2)
df_measurement_features_val_derivative.columns = ['exp_id'] + feature_names_derivative
df_measurement_features_val = pd.merge(df_measurement_features_val_orignal, df_measurement_features_val_derivative, on='exp_id', how='inner')
df_measurement_features_val_ratio = calculate_all_feature_product(df_measurement_features_val)
df_measurement_features_val = pd.merge(df_measurement_features_val, df_measurement_features_val_ratio, on='exp_id', how='inner')
df_measurement_features_val = pd.merge(df_measurement_features_val, df[['exp_id', 'CB', 'DCB', 'TOL', 'annealing_temperature', 'E 00',	'E 01',	'E 02',	'00/01',	'Relative Anion Signal',	'Bleaching %',	'polaron ratio','conductivity']], on='exp_id', how='inner')
df_measurement_features_val = df_measurement_features_val[data_driven_features + expert_features + ['exp_id', 'conductivity']]


# Test data
df_measurement_features_test_orignal = calculate_features_for_boundaries( optimal_boundaries, measurement_csv_files_test, spectra_axis='energy',  order=0)
df_measurement_features_test_orignal.columns = ['exp_id'] + feature_names
df_measurement_features_test_derivative = calculate_features_for_boundaries( optimal_boundaries, measurement_csv_files_test, spectra_axis='energy',  order=2)
df_measurement_features_test_derivative.columns = ['exp_id'] + feature_names_derivative
df_measurement_features_test = pd.merge(df_measurement_features_test_orignal, df_measurement_features_test_derivative, on='exp_id', how='inner')
df_measurement_features_test_ratio = calculate_all_feature_product(df_measurement_features_test)
df_measurement_features_test = pd.merge(df_measurement_features_test, df_measurement_features_test_ratio, on='exp_id', how='inner')
df_measurement_features_test = pd.merge(df_measurement_features_test, df[['exp_id', 'CB', 'DCB', 'TOL', 'annealing_temperature', 'E 00', 'E 01',	'E 02',	'00/01',	'Relative Anion Signal',	'Bleaching %',	'polaron ratio', 'conductivity']], on='exp_id', how='inner')
df_measurement_features_test = df_measurement_features_test[data_driven_features + expert_features + ['exp_id', 'conductivity']]

X_train = df_measurement_features_train.drop(columns=['exp_id', 'conductivity'])  
y_train = df_measurement_features_train['conductivity'].values

X_val = df_measurement_features_val.drop(columns=['exp_id', 'conductivity'])
y_val = df_measurement_features_val['conductivity'].values

X_test = df_measurement_features_test.drop(columns=['exp_id', 'conductivity'])
y_test = df_measurement_features_test['conductivity'].values

model_5 = GradientBoostingRegressor(loss ='squared_error', learning_rate=0.1, n_estimators=150, min_samples_leaf=5, random_state=42)
y_pred_train, y_pred_val, y_pred_test, _, _, _, _, _, _,_, _,_,_, _,_, _,_,_ = calculate_statistics(model_5, X_train, y_train, X_val, y_val, X_test, y_test)

plt.figure(figsize=(6.2, 6.2))
plt.scatter(y_train, y_pred_train, color='black', alpha=0.7, label='Train')
plt.scatter(y_val, y_pred_val, color='blue', alpha=0.7, label='Val')
plt.scatter(y_test, y_pred_test, color='red', alpha=0.7, label='Test')
plt.plot([min(np.concatenate((y_train, y_test))), max(np.concatenate((y_train, y_test)))],
    [min(np.concatenate((y_train, y_test))), max(np.concatenate((y_train, y_test)))],
    color='green', linestyle='--')
plt.xlabel('True Conductivity (S/cm)', fontsize=24)
plt.ylabel('Predicted Conductivity(S/cm)', fontsize=24)
plt.legend(fontsize=24)
plt.tight_layout()
plt.xticks(fontsize=24)
plt.yticks(fontsize=24)
plt.show()


SHAPLEY Feature Selection for QSPR 5

In [ ]:
explainer = shap.Explainer(model_5, X_train)
shap_values = explainer(X_train)
# Summary plot
plt.rcParams.update({
    'font.size': 24,
    'axes.labelsize': 24,
    'xtick.labelsize': 24,
    'ytick.labelsize': 24,
    'legend.fontsize': 24,
    'figure.titlesize': 24
})

shap.summary_plot(shap_values, X_train, plot_type="bar", max_display=20, color='k', plot_size=(9,8), show=False)
plt.show()
plt.close()

shap.plots.beeswarm(shap_values, max_display=20, show = False, plot_size=(8,6))
plt.tight_layout()
plt.show()
plt.close()

mean_abs_shap = np.abs(shap_values.values).mean(axis=0)
shap_feature_importance = list(zip(X_test.columns, mean_abs_shap))
shap_feature_importance = sorted(shap_feature_importance, key=lambda x: x[1], reverse=True)
shap_ranked_features = [f[0] for f in shap_feature_importance]

r2_scores_shap = []
num_features_shap = []
mae_scores_shap = []

for i in range(1, len(shap_ranked_features) + 1):
    selected_features = shap_ranked_features[:i]
    print(f"Using top {i} SHAP features: {selected_features}")
    X_train_subset = X_train[selected_features]
    X_val_subset = X_val[selected_features]
    X_test_subset = X_test[selected_features]
    y_pred_train, y_pred_val, y_pred_test, _, r2_val, _, _, rmse_val, _,_, mae_val,_,_, _,_, _,_,_ = calculate_statistics(model_5, 
                                                                                                        X_train_subset, y_train, 
                                                                                                        X_val_subset, y_val, 
                                                                                                        X_test_subset, y_test)
    
    
    r2_scores_shap.append(r2_val)
    mae_scores_shap.append(mae_val)
    num_features_shap.append(i)

    if i == 18: # Min MAE at 18 features
        plt.figure(figsize=(6.5, 6.5))
        plt.scatter(y_train, y_pred_train, color='black', alpha=0.7, label='Train')
        plt.scatter(y_val, y_pred_val, color='blue', alpha=0.7, label='Val')
        plt.scatter(y_test, y_pred_test, color='red', alpha=0.7, label='Test')
        plt.plot([min(np.concatenate((y_train, y_test))), max(np.concatenate((y_train, y_test)))],
            [min(np.concatenate((y_train, y_test))), max(np.concatenate((y_train, y_test)))],
            color='green', linestyle='--')
        plt.xlabel('True Conductivity (S/cm)', fontsize=22)
        plt.ylabel('Predicted Conductivity(S/cm)', fontsize=24)
        plt.legend(fontsize=24)
        plt.tight_layout()
        plt.xticks(fontsize=24)
        plt.yticks(fontsize=24)
        plt.show()
    print("-------------------------------------------------------------")

#Plot R² vs. number of SHAP-ranked features
plt.figure(figsize=(6, 6))
plt.plot(num_features_shap, mae_scores_shap, marker='o', color='k')
plt.ylabel('MAE (Val)', fontsize=24)
plt.xlabel('Number of Features', fontsize=24)
plt.xticks([0, 5, 10, 15, 20],fontsize=24)
plt.yticks(fontsize=24)
plt.tight_layout()
plt.grid(True, linestyle='--', alpha=0.5)
plt.show()


